In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [5]:
output_model_file = 'posnega_model/pytorch_model.bin'
output_config_file = 'posnega_model/config.json'
output_vocab_file = 'posnega_model/vocab.txt'

config = BertConfig.from_json_file(output_config_file)
model = BertForSequenceClassification(config)
state_dict = torch.load(output_model_file)
model.load_state_dict(state_dict)
tokenizer = BertTokenizer(output_vocab_file, do_lower_case=False)

In [9]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [10]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [11]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [24]:
model = model.to(device)
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [184]:
logits = test_sentences(['그럼에도 불구하고, SK팬이라면 허망할 수밖에 없는 완패였다'])

print(logits)
print(np.argmax(logits))

[[ 1.0681297  -0.93374485]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [195]:
# fine-tuning
train = pd.read_csv('dataset/finetuning_aihub_model.csv', encoding='cp949')
train

,Unnamed: 0,sentence,label
0,0,"[포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...",1
1,1,한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...,1
2,2,SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...,1
3,3,신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...,0
4,4,코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...,1
...,...,...,...
26114,31382,""" -SK와이번스 팬들에게 한마디 ""이곳에 와서 너무 기쁘다",1
26115,31383,정상적으로 합류하기까지 2주간 격리를 해야 한다는 사실이 너무 아쉽다,0
26116,31384,하루 빨리 야구장에 나가 경기에 출전하고 싶고 팬들을 만나고 싶다,1
26117,31385,SK와이번스를 위해 뛰게 된다는 것이 너무 행복하다,1


In [196]:
# 리뷰 문장 추출
sentences = train['sentence']
sentences[:10]

0    [포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전...
1      한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 ...
2     SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내...
3      신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야...
4     코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만...
5               종전 가장 늦게 개막한 시즌은 1995년으로, 4월 15일에 출발했다
6      공교롭게도 39년 역사상 가장 오래 기다렸던 개막전은 39년 역사상 가장 짧게 ...
7                         이날 인천 경기의 공식 소요 시간은 단 2시간 6분
8                         오후 2시 1분에 개시해 오후 4시 7분에 종료됐다
9     이는 역대 개막전 최단 시간 신기록으로, 종전 최단 시간이었던 2000년 4월 5...
Name: sentence, dtype: object

In [197]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

["[CLS] [포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전[엑스포츠뉴스 인천, 조은혜 기자] 역대 가장 늦게 시작한 KBO리그, 한화 이글스와 SK 와이번스의 공식 개막전은 역대 가장 짧은 시간 펼쳐졌다 [SEP]",
 '[CLS]   한화와 SK는 5일 인천SK행복드림구장에서 2020 신한은행 SOL KBO리그 정규시즌 개막전을 펼쳤다 [SEP]',
 '[CLS]  SK가 2018년 한국시리즈 우승팀이었기 때문에 SK의 홈 경기인 이 경기가 국내는 물론 해외에서도 주목하는 공식 개막전이었다 [SEP]',
 '[CLS]   신종 코로나바이러스 감염증(코로나19) 여파로 개막이 연기되면서 5월 5일에서야 포문을 연 프로야구 [SEP]',
 '[CLS]  코로나19 확산세로 개막을 엄두조차 내지 못하고 있는 다른 나라에 비하면 빠르지만, 5월 5일 개막은 역대 KBO리그 가장 늦은 개막이다 [SEP]',
 '[CLS]  종전 가장 늦게 개막한 시즌은 1995년으로, 4월 15일에 출발했다 [SEP]',
 '[CLS]   공교롭게도 39년 역사상 가장 오래 기다렸던 개막전은 39년 역사상 가장 짧게 끝이 났다 [SEP]',
 '[CLS]  이날 인천 경기의 공식 소요 시간은 단 2시간 6분 [SEP]',
 '[CLS]  오후 2시 1분에 개시해 오후 4시 7분에 종료됐다 [SEP]',
 '[CLS]  이는 역대 개막전 최단 시간 신기록으로, 종전 최단 시간이었던 2000년 4월 5일 잠실 해태-두산전의 2시간 11분보다 6분이 빨랐다 [SEP]']

In [198]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [204]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] [포인트:컷] '2시간 6분 컷' 한화-SK, 역사상 가장 늦고 가장 짧았던 개막전[엑스포츠뉴스 인천, 조은혜 기자] 역대 가장 늦게 시작한 KBO리그, 한화 이글스와 SK 와이번스의 공식 개막전은 역대 가장 짧은 시간 펼쳐졌다 [SEP]
['[CLS]', '[', '포', '##인', '##트', ':', '컷', ']', "'", '2', '##시간', '6', '##분', '컷', "'", '한', '##화', '-', 'SK', ',', '역', '##사', '##상', '가장', '늦', '##고', '가장', '짧', '##았', '##던', '개', '##막', '##전', '[', '엑', '##스', '##포츠', '##뉴스', '인', '##천', ',', '조', '##은', '##혜', '기자', ']', '역', '##대', '가장', '늦', '##게', '시', '##작', '##한', 'KB', '##O', '##리그', ',', '한', '##화', '이', '##글', '##스', '##와', 'SK', '와', '##이', '##번', '##스의', '공식', '개', '##막', '##전', '##은', '역', '##대', '가장', '짧', '##은', '시', '##간', '펼', '##쳐', '##졌다', '[SEP]']


In [205]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    164,   9928,  12030,  15184,    131,   9804,    166,
          112,    123, 100699,    127,  37712,   9804,    112,   9954,
        18227,    118,  21275,    117,   9566,  12945,  14871,  22224,
         9047,  11664,  22224,   9717, 119118,  23990,   8857, 118907,
        16617,    164,   9560,  12605,  90578,  90947,   9640,  38631,
          117,   9678,  10892, 119437,  60886,    166,   9566,  14423,
        22224,   9047,  14153,   9485,  38709,  11102,  47971,  11403,
        44689,    117,   9954,  18227,   9638, 118663,  12605,  12638,
        21275,   9590,  10739,  35465,  49319,  73844,   8857, 118907,
        16617,  10892,   9566,  14423,  22224,   9717,  10892,   9485,
        18784,   9925, 119265,  32855,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [206]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [207]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101, 12370, 11261,   130, 19855, 11903,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [213]:
# 배치 사이즈
batch_size = 8

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [214]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [215]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [216]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [219]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.enabled = False

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
       
        # Forward 수행                
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyError: 'exp_avg_sq'

In [220]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(),
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   100  of    817.    Elapsed: 0:00:11.
  Batch   200  of    817.    Elapsed: 0:00:22.
  Batch   300  of    817.    Elapsed: 0:00:33.
  Batch   400  of    817.    Elapsed: 0:00:44.
  Batch   500  of    817.    Elapsed: 0:00:55.
  Batch   600  of    817.    Elapsed: 0:01:06.
  Batch   700  of    817.    Elapsed: 0:01:17.
  Batch   800  of    817.    Elapsed: 0:01:28.

Accuracy: 0.53
Test took: 0:01:29
